***This notebook contains a script that sends Student Contracts to Students for signing***

**Add local library to path**

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    sys.path.append(module_path + '/local_library')

***Import Libraries***

In [2]:
import pandas as pd
import requests
import json
from datetime import date
from local_library import import_worksheet
from local_library import export_worksheet
from local_library import send_bulk_signrequest

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=1017190226189-f1d5s7cpjrj54u2rqk1ufh9pevguqoap.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


**Import Data**

In [17]:
#import 2020 data worksheet
cohort_df = import_worksheet("Cohort 2020 Data", "Cohort 2020 Data")
cohort_df = cohort_df[cohort_df['Active / Not active/Early absorption'] == 'Active']

**Create Learner FullName**

In [18]:
cohort_df['learner_fullname'] = cohort_df['Firstname'] + ' ' + cohort_df['Surname']

**Create Learner Initials**

In [19]:
cohort_df["learner_initials"] = [name.split()[0][0] + "." + name.split()[1][0] for name in cohort_df['Firstname'] + ' ' + cohort_df['Surname']]


**Create Learner Passport**

In [20]:
cohort_df["learner_passport"] = [passport if len(str(passport)) != 13 else " " for passport in cohort_df["ID Number/ Passport Number"]]


**Create Learner Equity Code**

In [11]:
cohort_df['learner_equity'] = cohort_df['Ethnicity'].map({'White':'WH', 'white':'WH', 'Coloured':'BC', 'coloured':'BC', 
                                                     'black':'BA', 'Black':'BA', 'Asian':'BI', 'indian':'BI','Indian':'BI'})

**Create Learner Learnership**

In [35]:
cohort_df['learner_nationality'] = cohort_df['Nationality'].map({'South African':'SA', 'Zimbabwean':'ZIM', 'Congolese':'ROA',
                                                                'Dutch':'EUR', 'Nigerian':'ROA'})

**Create Learner Citizenship**

In [ ]:
cohort_df['Residential Status (Citizen / Permanent Resident/Asylum Seeker/Work Permit/Study Permit)'] = cohort_df['Residential Status (Citizen / Permanent Resident/Asylum Seeker/Work Permit/Study Permit)'].fillna('Uknown')
cohort_df['learner_citizenship'] = cohort_df['Residential Status (Citizen / Permanent Resident/Asylum Seeker/Work Permit/Study Permit)'].map({"Unknown":"U", "Citizen":"SA",
                                                                                  "Permanent Resident":"O", "Dual":"D"})

**Create Learner Langauge**

In [37]:
cohort_df['learner_language'] = cohort_df['Home Language'].map({'English':'ENG', 'Afrikaans':'AFR', 'Sepedi [also known as Northern Sotho / Sesotho sa Lebowa]':'SEP',
       'tshiVenda':'TSH', 'isiZulu':'ZUL', 'isiXhosa':'XHO', 'seTswana':'SET', 'xiTsonga':'XIT',
       'isiNdebele':'NDE', 'seSotho':'SES', 'Other':'OTH', 'siSwati':'SWA', 'Sesotho':'SES'})


**Create Learner Socio Economic Status**

In [38]:
cohort_df["learner_socioeconomic"] = "01"

**Create Learner Disability**

In [44]:
cohort_df['Disability'] = cohort_df['Disability'].fillna('Uknown')
cohort_df['learner_disability'] = cohort_df['Disability'].map({'No':'N', 'no':'N', 'Yes':'09', 'None':'N', 'yes':'09', 'Yes - specwearer':'01', 'N':'N', 'ADD':'09',
       'Visual impairment - spec wearer':'02', 'Y - specwearer':'02', 'Visual impairment - Spec wearer -':'02', 'Uknown':'U', 'Aspergers / Autistic':'06',
       'Patella Alta':'04', 'yes - specwearer':'02', '\\N':'N', 'Y- specwearer':'02'})


**Create Province Code**

In [47]:
cohort_df['learner_province'] = cohort_df['Municipality'].map({'Cape Town Metro':'2', 'Johannesburg Metro':'8', 'Ekurhuleni Metro':'8',
       'Stellenbosch':'2', 'Tshwane Metro':'8', 'Mogale City':'8', 'Blouberg':'10',
       'Midvaal':'8', 'Drakenstein':'2', 'UMlazi municipality':'6', 'Molopo':'7',
       'Overberg':'2', 'Maluti a Phofung':'5', 'West Coast':'2'})

**Create Learner Designation**

In [48]:
cohort_df["learner_designation"] = "Learner"

**Create Learner Company**

In [49]:
cohort_df['learner_company'] = 'WeThinkCode_'

**Create Learner Title**

In [51]:
cohort_df['learner_title'] = ['Mr.' if gender.lower() == 'male' else 'Ms.' for gender in cohort_df['Gender']]

**Create Learner Contact**

In [53]:
cohort_df['learner_contact'] = 'Larissa Brand'

**Create Learner Contact Phone**

In [54]:
cohort_df['learner_contact_phone'] = '082 774 9982'

**Fix Learner Phone**

In [56]:
def convert_phone_to_string(target_df, column):
    """
    Converts phone number column to a string type.
    
    Args:
        target_df: The dataframe containing the column to be converted.
        column: The column to be converted.
        
    Returns:
        N/A
    """
    
    #convert to string
    target_df[column] = target_df[column].astype(str)
    
    #convert to float
    target_df[column] = target_df[column].astype(float)
    
    #convert to int
    target_df[column] = target_df[column].astype(int)
    
    #convert to string
    target_df[column] = target_df[column].astype(str)
    
#convert phone to string
convert_phone_to_string(cohort_df,"Mobile Number")

**Rename Columns**

In [57]:
cohort_df = cohort_df.rename(columns={"ID Number/ Passport Number":"learner_id", "Firstname":"learner_name", "Surname":"learner_surname", "Birth Date":"learner_birthdate",
                            "Gender":"learner_gender", "Home Address":"learner_postal_address", "Physical Address while studying":"learner_home_address",
                            "Mobile Number":"learner_phone", "Student Email":"learner_email"})


**Subset Columns**

In [58]:
#subset columns
column_name = ["learner_name", "learner_surname",'learner_fullname', 'learner_initials',
               'learner_passport','learner_equity', 'learner_nationality', 'learner_citizenship',
               'learner_language', 'learner_socioeconomic', 'learner_disability','learner_province', 'learner_designation',
              "learner_id", "learner_company", "learner_title", "learner_birthdate", "learner_gender", "learner_postal_address",
              "learner_home_address", "learner_phone", "learner_email", "learner_contact", "learner_contact_phone"]

cohort_df = cohort_df[column_name]

KeyError: "['learner_equity'] not in index"

***Local Functions Library***

In [34]:
def convert_phone_to_string(target_df, column):
    """
    Converts phone number column to a string type.
    
    Args:
        target_df: The dataframe containing the column to be converted.
        column: The column to be converted.
        
    Returns:
        N/A
    """
    
    #convert to string
    target_df[column] = target_df[column].astype(str)
    
    #convert to float
    target_df[column] = target_df[column].astype(float)
    
    #convert to int
    target_df[column] = target_df[column].astype(int)
    
    #convert to string
    target_df[column] = target_df[column].astype(str)
    
    
def create_rpl_dataframe():
    """
    Creates a dataframe for populating rpl registration documents
    
    Args:
        N/A
        
    Returns:
        rpl_form_df: A dataframe for populating rpl registration documents
    """
    
    #import rpl seta registration form
    rpl_form_df = import_worksheet("SETA Registration Form (Responses)", "Form Responses 1")
    
    #create learner fullname
    rpl_form_df['learner_fullname'] = rpl_form_df['Learner Name'] + " " + rpl_form_df['Learner Surname']
    
    #create learner_initials
    rpl_form_df["learner_initials"] = [name.split()[0][0] + "." + name.split()[1][0] for name in rpl_form_df['Learner Name'] + " " + rpl_form_df['Learner Surname']]
    
    #create learner passport
    rpl_form_df["learner_passport"] = [passport if len(str(passport)) != 13 else " " for passport in rpl_form_df["Learner ID"]]
    
    #create learner equity code
    rpl_form_df["learner_equity"] = rpl_form_df["Ethnicity"].map({"Black African": "BA", "Black Coloured":"BC",
                                                                "Black Indian/Asian":"BI", "White":"WH", "Uknown":"U"})
    
    #create learner nationality
    rpl_form_df["learner_nationality"] = rpl_form_df["Nationality"].map({"Unspecified":"U", "South Africa":"SA", "SADC":"SDC",
                                                                    "Namibia":"NAM", "Botswana":"BOT", "Zimbabwe":"ZIM", "Angola":"ANG",
                                                                    "Mozambique":"MOZ", "Lesotho":"LES", "Lesotho":"LES", "Swaziland":"SWA",
                                                                    "Malawi":"MAL", "Zambia":"ZAM", "Mauritius":"MAU", "Tanzania":"TAN",
                                                                    "Seychelles":"SEY", "Zaire":"ZAI", "Rest of Africa":"ROA",
                                                                     "European countries":"EUR", "Asian countries":"AIS",
                                                                     "North American countries":"NOR", "Central and South American countries":"SOU",
                                                                    "Australia and New Zealand":"AUS", "Other and rest of Oceania":"OOC"})
    
    #create learner citizenship
    rpl_form_df["learner_citizenship"] = rpl_form_df["Citizen/Residence status"].map({"Unknown":"U", "South Africa":"SA",
                                                                                  "Other":"O", "Dual":"D"})
    
    #create learner language
    rpl_form_df["learner_language"] = rpl_form_df["Home Language"].map({"English":"ENG", "Afrikaans":"AFR", "Other":"OTH",
                                                                   "sePedi":"SEP", "seSotho":"SES", "seTswana":"SET",
                                                                   "siSwati":"SWA", "tshiVenda":"TSH", "Uknown":"U",
                                                                   "isiXhosa":"XHO", "xiTsonga":"XIT", "isiZulu":"ZUL",
                                                                   "siNdebele":"NDE"})
    
    #create learner socio economic status
    rpl_form_df["learner_socioeconomic"] = "01"
    
    #create learner disability
    rpl_form_df["learner_disability"] = rpl_form_df["Disability Status"].map({"None":"N", "Sight (even with glasses)":"01", "Hearing (even with hearing aid)":"02",
                                                                         "Communication (talking, listening)":"03", "Physical (moving, standing, grasping)":"04",
                                                                         "Intellectual (difficulties in learning)":"05", "Emotional (behavioural or psychological)":"06",
                                                                         "Multiple":"07", "Disabled but unspecified":"09", "Uknown":"U"})
    
    #create province code
    rpl_form_df["learner_province"] = rpl_form_df["Province"].map({"Gauteng":"8", "Western Cape":"2", "Eastern Cape":"3", "Northern Cape":"4",
                                                               "Free State":"5", "Limpopo":"10", "Kwazulu-Natal":"6", "Mpumalanga":"9",
                                                              "North West":"7", "Undefined":"1"})
    
    #convert phone to string
    convert_phone_to_string(rpl_form_df,"Learner Phone")
    
    #create learner designation
    rpl_form_df["learner_designation"] = "Learner"
    
    #rename columns
    rpl_form_df = rpl_form_df.rename(columns={"Learner ID":"learner_id", 
                                "Which company did(do) you work for? The answer is  WeThinkCode_ if you are a student or you are unemployed.":"learner_company",
                                "Learner Title":"learner_title", "Learner Name":"learner_name", "Learner Surname":"learner_surname", "Birth Date":"learner_birthdate",
                                "Gender":"learner_gender", "Postal Address (If different from Home Address)":"learner_postal_address", "Home Address":"learner_home_address",
                                "Learner Phone":"learner_phone", "Learner Email":"learner_email", "Full name of contact person at company of employment.":"learner_contact",
                                "Phone number of contact person at company of employment.":"learner_contact_phone"})
    
    #subset columns
    column_name = ["learner_name", "learner_surname",'learner_fullname', 'learner_initials',
                   'learner_passport','learner_equity', 'learner_nationality', 'learner_citizenship',
                   'learner_language', 'learner_socioeconomic', 'learner_disability','learner_province', 'learner_designation',
                  "learner_id", "learner_company", "learner_title", "learner_birthdate", "learner_gender", "learner_postal_address",
                  "learner_home_address", "learner_phone", "learner_email", "learner_contact", "learner_contact_phone"]
    
    #replace all null values
    rpl_form_df.fillna('  ', inplace=True)
    
    return rpl_form_df[column_name]



**Create Contract DataFrame**

In [35]:
rpl_df = create_rpl_dataframe()

***Send POE Instructions Document***

In [37]:
#create target columns
poe_instructions_columns = ['learner_fullname', 'learner_id', 'learner_initials', 'learner_company', 'learner_email']

#create template column
rpl_df['poe_instructions_template_column'] = "feb67241-52f8-456c-917e-bdd0526619ee"

#send poe instructions through signrequests in bulk
#send_bulk_signrequest(rpl_df, poe_instructions_columns, 'poe_instructions_template_column')

Signer: empoweryourself36@gmail.com , Status: 201
Signer: solomonyedidia@gmail.com , Status: 201


**Send Learner Contract Document**

In [38]:
#create target columns
learner_contract_columns = ['learner_title', 'learner_name', 'learner_surname', 'learner_birthdate', 'learner_id', 'learner_passport',
                           'learner_equity', 'learner_nationality', 'learner_gender', 'learner_citizenship', 'learner_language',
                           'learner_socioeconomic', 'learner_disability', 'learner_home_address', 'learner_postal_address',
                           'learner_province', 'learner_phone', 'learner_email', 'learner_company', 'learner_contact', 'learner_contact_phone',
                           'learner_designation']

#create template column
rpl_df['learner_contract_template_column'] = '900be1ba-0b96-4386-a904-fb67069f168e'

#send learner contract through signrequests in bulk
#send_bulk_signrequest(rpl_df, learner_contract_columns, 'learner_contract_template_column')

Signer: empoweryourself36@gmail.com , Status: 201
Signer: solomonyedidia@gmail.com , Status: 201


**Send Learner Assessment Contract**

In [40]:
#create target columns
learner_assessment_contract_columns = ['learner_fullname', 'learner_id', 'learner_email']

#create template column
rpl_df['learner_assessment_contract_template_column'] = '0cb728c3-790c-4ea3-9a38-33d9c06d6547'

#send learner assessment contract through signrequests in bulk
#send_bulk_signrequest(rpl_df, learner_assessment_contract_columns, 'learner_assessment_contract_template_column')

Signer: empoweryourself36@gmail.com , Status: 201
Signer: solomonyedidia@gmail.com , Status: 201


**Send Learner Declaration of Authenticity**

In [41]:
#create target columns
learner_authenticity_documents_columns = ['learner_fullname', 'learner_id', 'learner_initials', 'learner_name', 'learner_email']

#create template column
rpl_df['learner_authenticity_document_template_column'] = '8492f73a-4de6-413f-8bfe-76b2556fce89'

#send learner authenticity document through signrequests in bulk
#send_bulk_signrequest(rpl_df, learner_authenticity_documents_columns, 'learner_authenticity_document_template_column')

Signer: empoweryourself36@gmail.com , Status: 201
Signer: solomonyedidia@gmail.com , Status: 201


**Send Learner PreAssessment Meeting Document**

In [42]:
#create target columns
learner_preassessment_document_columns = ['learner_fullname', 'learner_id', 'learner_contact', 'learner_email']

#create template column
rpl_df['learner_preassessment_document_template_column'] = '57d91620-97ad-462a-a32a-ed1f77934156'

#send learner preassessment meeting document through signrequests in bulk
#send_bulk_signrequest(rpl_df, learner_preassessment_document_columns, 'learner_preassessment_document_template_column')

Signer: empoweryourself36@gmail.com , Status: 201
Signer: solomonyedidia@gmail.com , Status: 201


***Create Test Dataframe***

In [31]:
test_df = rpl_df.groupby(['learner_preassessment_document_template_column']).head(1).groupby('learner_preassessment_document_template_column').head(1).copy()
test_df['learner_email'] = 'mufaro@thoughtquest.co.za'
send_bulk_signrequest(test_df, learner_preassessment_document_columns, 'learner_preassessment_document_template_column')

Signer: mufaro@thoughtquest.co.za , Status: 201
